### Research Process 


#### Init Libraries

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
import os

#### Load Dataset and split into Features and Label

In [ ]:
#load data
df=pd.read_csv("breast-cancer-wisconsin-data/data.csv")
#drop irelevent columns for the classification
df = df.drop(columns=['Unnamed: 32', 'id'])
# rearange the data for X - featuers and Y leabels 
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

#### Encode Labels

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

#### The mapping of the labels

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

#### Splitting the data into Train, Test and Valid datasets

In [ ]:
#split the data
from sklearn.model_selection import train_test_split
XData_train_val, XData_test, ydata_train_val, ydata_test = train_test_split(X, y, test_size=0.2, random_state=42)
XData_train, XData_val, ydata_train, ydata_val = train_test_split(XData_train_val, ydata_train_val, test_size=0.2, random_state=42)

#### Scale the data

In [ ]:
scaler = RobustScaler()
X_train = scaler.fit_transform(XData_train)
X_test = scaler.transform(XData_test)
X_val = scaler.transform(XData_val)

X_train_val = XData_train_val.values

y_train = ydata_train
y_test = ydata_test
y_val = ydata_val

y_train_val = ydata_train_val

#### Handling the imbalance between the classes

In [ ]:
#handling imbalance data 
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
#the class weight is done only on the train data to impact the learning process and to evaluete beter the model proformence
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

as predicted the minorty class 'M'=1 gets higher weight of 1.368

### Neural Networks

##### Init Libraries

In [ ]:
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.regularizers import l1, l2
#from keras.layers import Dropout, Flatten, BatchNormalization
from keras import models
from keras import layers
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks
from keras import utils

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf

##### Function Helpers

In [ ]:
def model_result(model):
    print('================================================================================')
    print(f'Evaluation Report for Model: {model.name}')

    # accuracy
    result = model.evaluate(X_test, y_test, verbose=0)        
    print(f'Loss Value: {result[0]:.3f}, Accuracy: {result[1]*100:.3f}%')

    # confusion matrix
    y_prob = model.predict(X_test, verbose=0)
    y_pred = np.round(y_prob).astype(int)    
    cm_test = confusion_matrix(y_test, y_pred)    
    #sns.heatmap(cm_test,annot=True)
    cm_disp = ConfusionMatrixDisplay(cm_test, display_labels=le.classes_)
    cm_disp.plot()
    plt.show()

    # classification report
    report = classification_report(y_test, y_pred, target_names=le.classes_)
    print(report)

    print('Model Summary:')
    print(model.summary())
    print('================================================================================')

In [ ]:
def model_history(history, param=None):
    plt.figure(figsize=(15,6))

    val_loss_min_pos = np.argmin(history.history['val_loss'])

    # Plotting the training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss', color='#8502d1')
    plt.plot(history.history['val_loss'], label='Validation Loss')

    if param is None:
        plt.title('Train and Validation Loss')
    else:
        plt.title(f'Train and Validation Loss on {param}')

    plt.plot(val_loss_min_pos, history.history['val_loss'][val_loss_min_pos], 'r*', label='Min Validation Loss')
    plt.text(val_loss_min_pos, history.history['val_loss'][val_loss_min_pos], f'({val_loss_min_pos}, {history.history['val_loss'][val_loss_min_pos]:.3f})', va='bottom')
    plt.legend()

    # Plotting the training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy', color='#8502d1')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(val_loss_min_pos, history.history['val_accuracy'][val_loss_min_pos], 'r*', label='Validation Accuracy @ Min Validation Loss')
    plt.text(val_loss_min_pos, history.history['val_accuracy'][val_loss_min_pos], f'({val_loss_min_pos}, {history.history['val_accuracy'][val_loss_min_pos]:.3f})', va='bottom')

    if param is None:
        plt.title('Train and Validation Accuracy')
    else:
        plt.title(f'Train and Validation Accuracy on {param}')
        
    plt.legend()

    plt.show()    

In [ ]:
def proc_history(history):

    val_loss_min_pos = np.argmin(history.history['val_loss'])

    train_acc = history.history['accuracy'][val_loss_min_pos]
    train_loss = history.history['loss'][val_loss_min_pos]

    val_acc = history.history['val_accuracy'][val_loss_min_pos]
    val_loss = history.history['val_loss'][val_loss_min_pos]    

    val_recall = history.history['val_recall'][val_loss_min_pos]

    return {'Train Accuracy': train_acc, 'Train Loss': train_loss, 'Validation Accuracy': val_acc, 'Validation Loss': val_loss, 'Validation Recall': val_recall}

In [ ]:
def model_fit(model, optimizer=optimizers.RMSprop, learning_rate=0.001, epochNum=1000, batchSize=32, en_reduce_lr=False, en_early_stopping=True, pca=False, verbose="auto", Dataset=None):      
    
    
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True, verbose=0)
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=0)
    checkpoint_filepath = f'model_checkpoints/{model.name}_checkpoint.model.keras'
    model_checkpoint_callback = callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=0)
    
    callbacks_list = [model_checkpoint_callback]

    if en_reduce_lr:
        callbacks_list.append(reduce_lr)

    if en_early_stopping:
        callbacks_list.append(early_stopping)

    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy', metrics.Recall(name='recall')])
    if Dataset is None:
        history = model.fit(X_train, y_train, batch_size=batchSize, epochs=epochNum, validation_data=(X_val, y_val), class_weight=class_weight_dict, callbacks=callbacks_list, verbose=verbose)
    else:
        history = model.fit(Dataset['X_train'], Dataset['y_train'], batch_size=batchSize, epochs=epochNum, validation_data=(Dataset['X_val'], Dataset['y_val']), class_weight=class_weight_dict, callbacks=callbacks_list, verbose=verbose)

    model = models.load_model(checkpoint_filepath)

    return history, model

In [ ]:
def model_kfold(xtrain, ytrain, model_base, optimizer=optimizers.RMSprop,learning_rate=0.001, epochNum=1000, batchSize=32, en_reduce_lr=False, en_early_stopping=True, verbose="auto"):
    model = models.clone_model(model_base)
    fold_k = StratifiedKFold(n_splits = 5).split(xtrain, ytrain)
    
    result_df = pd.DataFrame(columns=['k', 'Train Accuracy', 'Train Loss' , 'Validation Accuracy', 'Validation Loss',  'Validation Recall'])
    for k , (train, valid) in enumerate(fold_k):

        scaler = RobustScaler()

        X_train = scaler.fit_transform(xtrain[train])
        X_val = scaler.transform(xtrain[valid])
        
        dataset = {'X_train': X_train, 'y_train': ytrain[train], 'X_val': X_val, 'y_val': ytrain[valid]}
        model.set_weights(model_base.get_weights())

        

        history, model = model_fit(model, optimizer=optimizer, learning_rate=learning_rate, epochNum=epochNum, batchSize=batchSize, en_reduce_lr=en_reduce_lr, en_early_stopping=en_early_stopping, Dataset=dataset, verbose=verbose)    
        
        proc_data = proc_history(history)

        new_row = {'k': k, **proc_data}

        result_df.loc[len(result_df)] = new_row

    return result_df

##### Build the model

In [ ]:
models_list = []

### SLP

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

#### Baseline

In [ ]:
CURRENT_MODEL = models.Sequential(name=f'SLP_1')

CURRENT_MODEL.add(layers.Input((30,)))
CURRENT_MODEL.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model = models.clone_model(CURRENT_MODEL)
model.set_weights(CURRENT_MODEL.get_weights())

##### Baseline - Results

In [ ]:
history, model = model_fit(model, epochNum=1000, batchSize=32, verbose=0)

proc_data = proc_history(history)

In [ ]:
model_history(history)

In [ ]:
model.set_weights(CURRENT_MODEL.get_weights())

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, batchSize=32, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
res_df

#### Optimizers

In [ ]:
df_optimizer = pd.DataFrame(columns=['Optimizer', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

for optimizer in [optimizers.Adam, optimizers.RMSprop, optimizers.SGD, optimizers.Adagrad]:
    print(f'------------------------------------------------------------------------------------------------------------------')
    print(f'Optimizer = {optimizer}')    
        
    model.set_weights(CURRENT_MODEL.get_weights())   

    learning_rate = 0.001
    if(optimizer == optimizers.SGD or optimizer==optimizers.Adagrad):
        learning_rate = 0.01

    history, model = model_fit(model, optimizer=optimizer, learning_rate = learning_rate, epochNum=2000, en_early_stopping=True, verbose=0)        
    proc_data = proc_history(history)
    
    new_row = {'Optimizer': optimizer, **proc_data}
    df_optimizer.loc[len(df_optimizer)] = new_row
    model_history(history, f'Optimizer = {optimizer.__name__}')


In [ ]:
df_optimizer

Conclusion: We can see in the graphs that the optimizers achieve similar results, while SGD and Adagrad take a large amount of Epochs to converge.

In [ ]:
model.set_weights(CURRENT_MODEL.get_weights())
ADAM_res_df = model_kfold(X_train_val, y_train_val, model, optimizer=optimizers.Adam, learning_rate=0.001, epochNum=1000, batchSize=32, verbose=0)

model.set_weights(CURRENT_MODEL.get_weights())
RMS_res_df = model_kfold(X_train_val, y_train_val, model, optimizer=optimizers.RMSprop, learning_rate=0.001, epochNum=1000, batchSize=32, verbose=0)

model.set_weights(CURRENT_MODEL.get_weights())
SGD_res_df = model_kfold(X_train_val, y_train_val, model, optimizer=optimizers.SGD, learning_rate=0.01, epochNum=1000, batchSize=32, verbose=0)

In [ ]:
print(f"Validation Loss Mean: {ADAM_res_df['Validation Loss'].mean()}, Validation Loss STD: {ADAM_res_df['Validation Loss'].std()}")
ADAM_res_df

In [ ]:
print(f"Validation Loss Mean: {RMS_res_df['Validation Loss'].mean()}, Validation Loss STD: {RMS_res_df['Validation Loss'].std()}")
RMS_res_df

In [ ]:
print(f"Validation Loss Mean: {SGD_res_df['Validation Loss'].mean()}, Validation Loss STD: {SGD_res_df['Validation Loss'].std()}")
SGD_res_df

Conclusion: We will take the best Optimizer as SGD, as it yields the lowest STD value in the Validation Loss.

In [ ]:
BEST_OPTIMIZER = optimizers.SGD

#### Learning-Rate

In [ ]:
df_res = pd.DataFrame(columns=['Learning Rate', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
    print(f'------------------------------------------------------------------------------------------------------------------')
    print(f'Learning Rate = {learning_rate}')    
        
    model.set_weights(CURRENT_MODEL.get_weights())   

    history, model = model_fit(model, optimizer=BEST_OPTIMIZER, learning_rate=learning_rate, epochNum=2000, en_early_stopping=True, verbose=0)
    proc_data = proc_history(history)
    
    new_row = {'Learning Rate': learning_rate, **proc_data}
    df_res.loc[len(df_res)] = new_row
    model_history(history, f'Learning Rate = {learning_rate}')


Conlusion: From the graphs above, we can infer that a learning rate of 0.1 and 0.01 yields the same results, however with a learning rate of 0.1 it yields the result 10 times faster, which means the model can handle a large learning rate at the begining of the training. <br>
In the next steps, we will include a learning rate scheduler, which will decrease the learning rate once it reaches a plateau.

In [ ]:
BEST_LEARNING_RATE = 0.1

#### Network Architecture

In [ ]:
nn_arch_df = pd.DataFrame(columns=['Model Name', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

##### NN 30x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### First Hidden Layer

##### NN 30x1x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x1x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(1, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x5x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x10x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x10x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x20x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x20x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x30x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

Concluion: We see that NN_30x30x1 is yields the best results for 1st hidden layer. <br>
We will check what is the best activation function for this layer.

#### Activation Functions

##### NN 30x30x1 - leaky_relu

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30-leaky_relu-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='leaky_relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x30x1 - silu

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30-silu-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='silu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x30x1 - elu

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30-elu-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='elu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history)
proc_data

##### NN 30x30x1 - tanh

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30-elu-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

Concluion: We see that relu yields the best results for 1st hidden layer activation function. <br>

##### Second Hidden Layer

##### NN 30x30x1x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x1x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history)
proc_data

##### NN 30x30x5x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x30x10x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x10x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history)
proc_data

##### NN 30x30x20x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x20x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x30x30x1

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x30x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history)
proc_data

Concluion: We see that NN_30x30x5x1 is yields the best results for 2st hidden layer. <br>
We will check what is the best activation function for this layer.

#### Activation Functions

##### NN 30x30x5x1 - leaky_relu

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x5-leaky_relu-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='leaky_relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x30x5x1 - silu

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x5-silu-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='silu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history)
proc_data

##### NN 30x30x5x1 - elu

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x5-elu-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='elu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

##### NN 30x30x5x1 - tanh

In [ ]:
# deterministic fitting   
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x5-tanh-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_arch_df.loc[len(nn_arch_df)] = new_row

model_history(history)
proc_data

Concluion: We see that relu yields the best results for 2st hidden layer activation function. <br>

In [ ]:
nn_arch_df

In [ ]:
nn_arch_df.sort_values(by='Validation Loss')

Conclusion: We can see that the Network Architecture of 30x30x5x1 yields the best results, while a leaky-relu or relu activation function in the second hidden layer yields the same results, thus we will remain with the default activation function. <br>
We will check this network with KFolds.

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  
        
model = models.Sequential(name=f'NN_30x30x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, epochNum=1000, batchSize=32, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
res_df

<b><u>Conclusion:</u></b> We can see that the 30x30x5x1 is the Network Architecture that yields the best and stable results. <br>
With that said, we will also still take the SLP, which is the basic Network Architecture (No hidden layers) into consideration.

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

BEST_MODEL = models.Sequential(name=f'NN_30x30x5x1')

BEST_MODEL.add(layers.Input((30,)))
BEST_MODEL.add(layers.Dense(30, activation='relu'))
BEST_MODEL.add(layers.Dense(5, activation='relu'))
BEST_MODEL.add(layers.Dense(1, activation='sigmoid'))

#### Batch-Size

In [ ]:
df_batch_size = pd.DataFrame(columns=['Batch Size', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

model = models.clone_model(BEST_MODEL)

for batch_size in [1, 2, 4, 8, 16, 32, 64, 128]:
    print(f'------------------------------------------------------------------------------------------------------------------')
    print(f'Batch Size = {batch_size}')    
        
    model.set_weights(BEST_MODEL.get_weights())   

    history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=batch_size, en_early_stopping=True, en_reduce_lr=True, verbose=0)
    proc_data = proc_history(history)
    
    new_row = {'Batch Size': batch_size, **proc_data}
    df_batch_size.loc[len(df_batch_size)] = new_row
    model_history(history, f'Batch Size = {batch_size}')


In [ ]:
for batch_size in [1, 2, 4, 8, 16, 32, 64, 128]:
    model.set_weights(BEST_MODEL.get_weights())
    batch_size_res_df = model_kfold(X_train_val, y_train_val, model, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, epochNum=1000, batchSize=batch_size, en_reduce_lr=True, en_early_stopping=True, verbose=0)
    print('----------------------------------------------------------------------------------------------------')
    print(f'Batch Size = {batch_size}')
    print(f"Validation Loss Mean: {batch_size_res_df['Validation Loss'].mean()}, Validation Loss STD: {batch_size_res_df['Validation Loss'].std()}")    
    print(f"Validation Recall Mean: {batch_size_res_df['Validation Recall'].mean()}, Validation Recall STD: {batch_size_res_df['Validation Recall'].std()}")    
    print(batch_size_res_df)    


Conclusion: As we can see from the results, BATCH_SIZE = 8 yielded the best results in the Validation Loss (mean), <br> 
As well as over the recall metric in the 5 folds. Thus, we will choose BATCH_SIZE = 8 as the best BATCH_SIZE.

In [ ]:
BEST_BATCH_SIZE = 8

## Handling Overfitting

### Dropout

In [ ]:
nn_drop_df = pd.DataFrame(columns=['Model Name', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

#### Input Layer Dropout

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30-D2-x30x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_drop_df.loc[len(nn_drop_df)] = new_row

model_history(history)
proc_data

#### First Layer Dropout

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30-D3-x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_drop_df.loc[len(nn_drop_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

#### Second Layer Dropout

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D3-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_drop_df.loc[len(nn_drop_df)] = new_row

model_history(history)
proc_data

#### First & Second Layer Dropout

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30-D3-x5-D3-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_drop_df.loc[len(nn_drop_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

#### KFold Assessment

In [ ]:
nn_drop_df

##### Input Layer Dropout Assessment

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30-D2-x30x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
res_df

##### First Layer Dropout Assessment

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30-D3-x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
res_df

##### Second Layer Dropout Assessment

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D3-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
res_df

##### First & Second Layer Dropout Assessment

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30-D3-x5-D3-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
res_df

#### Tune Dropout Rate = [0.1, 0.2, 0.3, 0.4, 0.5]

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

for rate in [0.1, 0.2, 0.3, 0.4, 0.5]:

    print('-----------------------------------------------------------------------------')
    print(f'Dropout Rate = {rate}')

    model = models.Sequential(name=f'NN_30x30x5-D{int(rate*10)}-x1')

    model.add(layers.Input((30,)))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dropout(rate))
    model.add(layers.Dense(1, activation='sigmoid'))

    res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
    print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
    print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
    print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
    print(res_df)

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_drop_df.loc[len(nn_drop_df)] = new_row

model_history(history)
proc_data

Conclusion: We chose the best Dropout Addition to the Network as a Dropout to the Second Layer only with a rate of 0.1. <br>
Thus, we will update our BEST_MODEL.

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

BEST_MODEL = models.Sequential(name=f'NN_30x30x5-D1-x1')

BEST_MODEL.add(layers.Input((30,)))
BEST_MODEL.add(layers.Dense(30, activation='relu'))
BEST_MODEL.add(layers.Dense(5, activation='relu'))
BEST_MODEL.add(layers.Dropout(0.1))
BEST_MODEL.add(layers.Dense(1, activation='sigmoid'))

### L2

In [ ]:
nn_l2_df = pd.DataFrame(columns=['Model Name', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

#### First Layer L2

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30_L2_0001x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu', kernel_regularizer=regularizers.L2(0.0001)))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_l2_df.loc[len(nn_l2_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

#### Second Layer L2

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5_L2_0001x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu', kernel_regularizer=regularizers.L2(0.0001)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_l2_df.loc[len(nn_l2_df)] = new_row

model_history(history)
proc_data

#### First & Second Layer L2

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30_L2_0001x5_L2_0001x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu', kernel_regularizer=regularizers.L2(0.0001)))
model.add(layers.Dense(5, activation='relu', kernel_regularizer=regularizers.L2(0.0001)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_l2_df.loc[len(nn_l2_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

In [ ]:
nn_l2_df

Conclusion: As we can see, adding the Regularization L2 didn't improve our model performance. <br>
Now we will check how L1 Regularization affects our model.

### L1

In [ ]:
nn_l1_df = pd.DataFrame(columns=['Model Name', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

#### First Layer L1

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30_L1_0001x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu', kernel_regularizer=regularizers.L1(0.0001)))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_l1_df.loc[len(nn_l1_df)] = new_row

model_history(history)
proc_data

#### Second Layer L1

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5_L1_0001x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu', kernel_regularizer=regularizers.L1(0.0001)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_l1_df.loc[len(nn_l1_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

#### First & Second Layer L1

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30_L2_0001x5_L1_0001x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu', kernel_regularizer=regularizers.L1(0.0001)))
model.add(layers.Dense(5, activation='relu', kernel_regularizer=regularizers.L1(0.0001)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_l1_df.loc[len(nn_l1_df)] = new_row

model_history(history)
proc_data

In [ ]:
nn_l1_df

Conclusion: As we can see, adding the Regularization L1 also didn't improve our model performance. <br>
Thus, we will not use L2 nor L1 Regularizations.

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

BEST_MODEL = models.Sequential(name=f'NN_30x30x5-D1-x1')

BEST_MODEL.add(layers.Input((30,)))
BEST_MODEL.add(layers.Dense(30, activation='relu'))
BEST_MODEL.add(layers.Dense(5, activation='relu'))
BEST_MODEL.add(layers.Dropout(0.1))
BEST_MODEL.add(layers.Dense(1, activation='sigmoid'))

### Bottleneck

In [ ]:
nn_bottleneck_df = pd.DataFrame(columns=['Model Name', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

#### Bottleneck between Input and First Layers

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x1x30x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(1, activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_bottleneck_df.loc[len(nn_bottleneck_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

#### Bottleneck between First and Second Layers

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x3x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(3, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_bottleneck_df.loc[len(nn_bottleneck_df)] = new_row

model_history(history)
proc_data

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x1x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_bottleneck_df.loc[len(nn_bottleneck_df)] = new_row

model_history(history, f'Model Name = {model.name}')
proc_data

#### Bottleneck as First Layer

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x1x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(1, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_bottleneck_df.loc[len(nn_bottleneck_df)] = new_row

model_history(history)
proc_data

In [ ]:
nn_bottleneck_df

#### KFold Assessment

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x1x5x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
res_df

Conclusion: We will not continue with bottleneck when we check other parameters, <br>
However, we will try to see the results of this model on the X_test dataset lateron, because we see that this model could have potential due to its stable graph.

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

BEST_MODEL = models.Sequential(name=f'NN_30x30x5-D1-x1')

BEST_MODEL.add(layers.Input((30,)))
BEST_MODEL.add(layers.Dense(30, activation='relu'))
BEST_MODEL.add(layers.Dense(5, activation='relu'))
BEST_MODEL.add(layers.Dropout(0.1))
BEST_MODEL.add(layers.Dense(1, activation='sigmoid'))

### Batch Normalization

In [ ]:
nn_batchnorm_df = pd.DataFrame(columns=['Model Name', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

#### Second Layer Batch Normalization

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_batchnorm_df.loc[len(nn_batchnorm_df)] = new_row

model_history(history, f'Batch Normalization = Second Layer')
proc_data

#### First Layer Batch Normalization

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0)
proc_data = proc_history(history)

new_row = {'Model Name': model.name, **proc_data}
nn_batchnorm_df.loc[len(nn_batchnorm_df)] = new_row

model_history(history, f'Batch Normalization = First Layer')
print(proc_data)

#### First Layer KFold

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
res_df

#### Second Layer KFold

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

model.add(layers.Input((30,)))
model.add(layers.Dense(30))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

res_df = model_kfold(X_train_val, y_train_val, model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_reduce_lr=True, en_early_stopping=True, verbose=0)
print(f"Validation Accuracy Mean: {res_df['Validation Loss'].mean()}, Validation Accuracy STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Loss Mean: {res_df['Validation Loss'].mean()}, Validation Loss STD: {res_df['Validation Loss'].std()}")    
print(f"Validation Recall Mean: {res_df['Validation Recall'].mean()}, Validation Recall STD: {res_df['Validation Recall'].std()}")    
res_df

Conclusion: This model didn't yield a higher validation score than our best model. <br>
For the rest of our parameters search, we will keep the previous best model.

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

BEST_MODEL = models.Sequential(name=f'NN_30x30x5-D1-x1')

BEST_MODEL.add(layers.Input((30,)))
BEST_MODEL.add(layers.Dense(30, activation='relu'))
BEST_MODEL.add(layers.Dense(5, activation='relu'))
BEST_MODEL.add(layers.Dropout(0.1))
BEST_MODEL.add(layers.Dense(1, activation='sigmoid'))

## Dimensionality Reduction

### PCA

In [ ]:
nn_pca_df = pd.DataFrame(columns=['PCA Components', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

In [ ]:
from sklearn import decomposition
pca_5 = decomposition.PCA(n_components=5)
pca_10 = decomposition.PCA(n_components=10)
pca_15 = decomposition.PCA(n_components=15)
pca_20 = decomposition.PCA(n_components=20)

#### PCA N_Components=5

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5x1')

X_train_pca_5 = pca_5.fit_transform(XData_train)
X_test_pca_5 = pca_5.transform(XData_test)
X_val_pca_5 = pca_5.transform(XData_val)

model.add(layers.Input((X_train_pca_5.shape[1],)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))



scaler_pca_5 = RobustScaler()
X_train_pca_5 = scaler_pca_5.fit_transform(X_train_pca_5)
X_test_pca_5 = scaler_pca_5.transform(X_test_pca_5)
X_val_pca_5 = scaler_pca_5.transform(X_val_pca_5)

dataset = {'X_train': X_train_pca_5, 'y_train': y_train, 'X_val': X_val_pca_5, 'y_val': y_val}
history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0, Dataset=dataset)
proc_data = proc_history(history)

new_row = {'PCA Components': 5, **proc_data}
nn_pca_df.loc[len(nn_pca_df)] = new_row

model_history(history, f'PCA Components = 5')
proc_data

#### PCA N_Components=10

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5x1')

X_train_pca_10 = pca_10.fit_transform(XData_train)
X_test_pca_10 = pca_10.transform(XData_test)
X_val_pca_10 = pca_10.transform(XData_val)

model.add(layers.Input((X_train_pca_10.shape[1],)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))



scaler_pca_10 = RobustScaler()
X_train_pca_10 = scaler_pca_10.fit_transform(X_train_pca_10)
X_test_pca_10 = scaler_pca_10.transform(X_test_pca_10)
X_val_pca_10 = scaler_pca_10.transform(X_val_pca_10)

dataset = {'X_train': X_train_pca_10, 'y_train': y_train, 'X_val': X_val_pca_10, 'y_val': y_val}
history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0, Dataset=dataset)
proc_data = proc_history(history)

new_row = {'PCA Components': 10, **proc_data}
nn_pca_df.loc[len(nn_pca_df)] = new_row

model_history(history, f'PCA Components = 10')
proc_data

#### PCA N_Components=15

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5x1')

X_train_pca_15 = pca_15.fit_transform(XData_train)
X_test_pca_15 = pca_15.transform(XData_test)
X_val_pca_15 = pca_15.transform(XData_val)

model.add(layers.Input((X_train_pca_15.shape[1],)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))



scaler_pca_15 = RobustScaler()
X_train_pca_15 = scaler_pca_15.fit_transform(X_train_pca_15)
X_test_pca_15 = scaler_pca_15.transform(X_test_pca_15)
X_val_pca_15 = scaler_pca_15.transform(X_val_pca_15)

dataset = {'X_train': X_train_pca_15, 'y_train': y_train, 'X_val': X_val_pca_15, 'y_val': y_val}
history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0, Dataset=dataset)
proc_data = proc_history(history)

new_row = {'PCA Components': 15, **proc_data}
nn_pca_df.loc[len(nn_pca_df)] = new_row

model_history(history, f'PCA Components = 15')
proc_data

#### PCA N_Components=20

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5x1')

X_train_pca_20 = pca_20.fit_transform(XData_train)
X_test_pca_20 = pca_20.transform(XData_test)
X_val_pca_20 = pca_20.transform(XData_val)

model.add(layers.Input((X_train_pca_20.shape[1],)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))



scaler_pca_20 = RobustScaler()
X_train_pca_20 = scaler_pca_20.fit_transform(X_train_pca_20)
X_test_pca_20 = scaler_pca_20.transform(X_test_pca_20)
X_val_pca_20 = scaler_pca_20.transform(X_val_pca_20)

dataset = {'X_train': X_train_pca_20, 'y_train': y_train, 'X_val': X_val_pca_20, 'y_val': y_val}
history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0, Dataset=dataset)
proc_data = proc_history(history)

new_row = {'PCA Components': 20, **proc_data}
nn_pca_df.loc[len(nn_pca_df)] = new_row

model_history(history, f'PCA Components = 20')
proc_data

In [ ]:
nn_pca_df

Conclusion: In our case, using PCA didn't help our model to learn better. <br>
We do see that even with fewer features, the model can achieve good enough results.

### Features Correlation

In [ ]:
nn_corr_df = pd.DataFrame(columns=['Threshold', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Validation Recall'])

As we saw in our EDA, there are some features that correlates to each other. Therefore, we can remove them in order to reduce the dimensionality of the problem and to make the model learn more easily. <br>
We will check how it affects the performance of our best model.

#### Correlation Matrix

In [ ]:
corr = XData_train.corr()
colormap = sns.diverging_palette(220, 10, as_cmap = True)
plt.figure(figsize=(14,14))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 8},
            cmap = colormap, linewidths=0.1, linecolor='white')
plt.title('Correlation of Breast Cancer Dataset Features', y=1.05, size=15)

We can see there are some correlated features in our dataset. <br>
We will evaluate how many features we remove for different thresholds.

Remove features that are above the defined threshold

In [ ]:
def get_corr_columns(X, threshold):

    # Compute the correlation matrix
    corr_matrix = X.corr()

    # Create a mask to identify upper triangle of the correlation matrix
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find index of features with correlation greater than the threshold
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]    
                                                                
    # Drop the highly correlated features
    X_reduced = X.drop(columns=to_drop)    

    return {'number of features to drop': len(to_drop), 'to_drop': to_drop, 'reduced': list(X_reduced.columns)}

df_corr_comp = pd.DataFrame(columns=['number of features to drop', 'to_drop', 'reduced'])
df_corr_comp.index.name = 'threshold'

thresholds = [0.85, 0.9, 0.95]

for threshold in thresholds:
    new_row = get_corr_columns(X, threshold)
    df_corr_comp.loc[threshold] = new_row

In [ ]:
df_corr_comp

We chose to check different high values of correlation in order to see how it influences the dimensons of the features. <br>
We assume that correlation value bigger than 0.9 means a high enough correlation for us to drop the feature. <br>

#### Correlation Threshold = 0.85

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

X_train_085 = XData_train.drop(columns=df_corr_comp.loc[0.85]['to_drop'])
X_test_085 = XData_test.drop(columns=df_corr_comp.loc[0.85]['to_drop'])
X_val_085 = XData_val.drop(columns=df_corr_comp.loc[0.85]['to_drop'])

model.add(layers.Input((X_train_085.shape[1],)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))



scaler_085 = RobustScaler()
X_train_085 = scaler_085.fit_transform(X_train_085)
X_test_085 = scaler_085.transform(X_test_085)
X_val_085 = scaler_085.transform(X_val_085)

dataset = {'X_train': X_train_085, 'y_train': y_train, 'X_val': X_val_085, 'y_val': y_val}
history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0, Dataset=dataset)
proc_data = proc_history(history)

new_row = {'Threshold': 0.85, **proc_data}
nn_corr_df.loc[len(nn_corr_df)] = new_row

model_history(history, f'Correlation Threshold = 0.85')
proc_data

#### Correlation Threshold = 0.90

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

X_train_090 = XData_train.drop(columns=df_corr_comp.loc[0.90]['to_drop'])
X_test_090 = XData_test.drop(columns=df_corr_comp.loc[0.90]['to_drop'])
X_val_090 = XData_val.drop(columns=df_corr_comp.loc[0.90]['to_drop'])

model.add(layers.Input((X_train_090.shape[1],)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))



scaler_090 = RobustScaler()
X_train_090 = scaler_090.fit_transform(X_train_090)
X_test_090 = scaler_090.transform(X_test_090)
X_val_090 = scaler_090.transform(X_val_090)

dataset = {'X_train': X_train_090, 'y_train': y_train, 'X_val': X_val_090, 'y_val': y_val}
history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0, Dataset=dataset)
proc_data = proc_history(history)

new_row = {'Threshold': 0.90, **proc_data}
nn_corr_df.loc[len(nn_corr_df)] = new_row

model_history(history, f'Correlation Threshold = 0.90')
proc_data

#### Correlation Threshold = 0.95

In [ ]:
os.environ['PYTHONHASHSEED']=str(0)
utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()  

model = models.Sequential(name=f'NN_30x30x5-D1-x1')

X_train_095 = XData_train.drop(columns=df_corr_comp.loc[0.95]['to_drop'])
X_test_095 = XData_test.drop(columns=df_corr_comp.loc[0.95]['to_drop'])
X_val_095 = XData_val.drop(columns=df_corr_comp.loc[0.95]['to_drop'])

model.add(layers.Input((X_train_095.shape[1],)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))



scaler_095 = RobustScaler()
X_train_095 = scaler_095.fit_transform(X_train_095)
X_test_095 = scaler_095.transform(X_test_095)
X_val_095 = scaler_095.transform(X_val_095)

dataset = {'X_train': X_train_095, 'y_train': y_train, 'X_val': X_val_095, 'y_val': y_val}
history, model = model_fit(model, epochNum=1000, optimizer=BEST_OPTIMIZER, learning_rate=BEST_LEARNING_RATE, batchSize=BEST_BATCH_SIZE, en_early_stopping=True, en_reduce_lr=True, verbose=0, Dataset=dataset)
proc_data = proc_history(history)

new_row = {'Threshold': 0.95, **proc_data}
nn_corr_df.loc[len(nn_corr_df)] = new_row

model_history(history, f'Correlation Threshold = 0.95')
proc_data

In [ ]:
nn_corr_df

Conclusion: In our case, dropping features that are correlated didn't help the model to learn better. <br>
We do see that even with fewer features, the model can achieve good enough results.